In [1]:
import numpy as np
from agents.random_agent import random_policy
from citylearn.citylearn import CityLearnEnv
import wandb
import random
import torch
import numpy as np
from collections import namedtuple, deque
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Constants:
    episodes = 3
    schema_path = './data/citylearn_challenge_2022_phase_1/schema.json'

def action_space_to_dict(aspace):
    """ Only for box space """
    return { "high": aspace.high,
             "low": aspace.low,
             "shape": aspace.shape,
             "dtype": str(aspace.dtype)
    }

def env_reset(env):
    observations = env.reset()
    action_space = env.action_space
    observation_space = env.observation_space
    building_info = env.get_building_information()
    building_info = list(building_info.values())
    action_space_dicts = [action_space_to_dict(asp) for asp in action_space]
    observation_space_dicts = [action_space_to_dict(osp) for osp in observation_space]
    obs_dict = {"action_space": action_space_dicts,
                "observation_space": observation_space_dicts,
                "building_info": building_info,
                "observation": observations }
    return obs_dict

In [3]:
env = CityLearnEnv(schema=Constants.schema_path)

In [4]:
print(env.action_space)

[Box(-1.0, 1.0, (1,), float32), Box(-1.0, 1.0, (1,), float32), Box(-1.0, 1.0, (1,), float32), Box(-1.0, 1.0, (1,), float32), Box(-1.0, 1.0, (1,), float32)]


In [5]:
print(env.observation_space[0].shape)
print(env.observation_space[1].shape)
print(env.observation_space[2].shape)
print(env.observation_space[3].shape)
print(env.observation_space[4].shape)


(28,)
(28,)
(28,)
(28,)
(28,)


###### 1 agent 28*5 5 -> good for starting
###### Try optimize the current models (Next Tuesday)  
###### First Hyperparameter tuning and then use the best hyperparam to change reward function changes(decide upon which kind of method to use)
###### List of possible reward function 200-300 epochs  
###### 5 agents (28*1) 1
###### 5 agents (28*5) 1
###### MAIRL

###### Sept30 

In [6]:
Transition_DDPG = namedtuple('Transition', ('state', 'next_state', 'action', 'reward', 'done'))
class DDPG_Memory(object):
    def __init__(self,capacity):
        """
        Replay buffer for DDPG
        INPUT  : state,next_state,action,reward and done
        OUTPUT : batch of off-polocy experience
        """
        self.memory = deque(maxlen=capacity)
        self.capacity = capacity

    def push(self, state, next_state, action, reward, done):
        self.memory.append(Transition_DDPG(state, next_state, action, reward, done))

    def sample(self, batch_size):
        transitions = random.sample(self.memory, batch_size)
        batch = Transition_DDPG(*zip(*transitions))
        return batch

    def __len__(self):
        return len(self.memory)


In [7]:
action = [([random.uniform(-1,1)]) for _ in range(5)]
print(action)

[[-0.8608503809017438], [0.7856104610265959], [-0.626287361884049], [0.7304368199592801], [-0.0751147526842082]]


In [8]:
env.reset()
action = [([random.uniform(-1,1)]) for _ in range(5)]
next_state, reward, done, _ = env.step(action)
metrics_t = env.evaluate()
metrics = {"price_cost": metrics_t[0], "emmision_cost": metrics_t[1]}
print(metrics)


{'price_cost': 1.3946967762483233, 'emmision_cost': 1.3744903942174411}


In [9]:

a = [i[0] for i in action]
print(a)

[-0.30880322261702986, 0.9471223677436154, -0.9593148921909942, -0.6975377997031114, 0.003272554528882532]
